# How to train simple bigram language models in practice

This notebook demonstrates word counting and bigram language models.

## The "Sam corpus"

Our first example uses the Green Eggs and Ham poem as a "corpus" from which we train a language model.

In [1]:
# we import the natural language toolkit
import nltk


# Here is our corpus, first as plain text
text = "I am Sam. Sam I am. I do not like green eggs and ham."
# ... and here it is cut up into sentences
text_sents = nltk.sent_tokenize(text)
# ... and cut up into words
text_words = nltk.word_tokenize(text)
# ... and cut up into sentences, each of which
# is cut up into words
text_sent_words = [ ]
for s in text_sents:
    text_sent_words.append(nltk.word_tokenize(s))
    
print("Corpus as a list of sentences:", text_sents)

Corpus as a list of sentences: ['I am Sam.', 'Sam I am.', 'I do not like green eggs and ham.']


We add sentence beginning and end markers, so that we can also count how often a word appears in the beginning or end of a sentence.

In [2]:
# this is a corpus of consecutive words, not a corpus
# of sentences
sam_corpus = []
for st in text_sent_words:
    sam_corpus = sam_corpus + ["<s>"] + st + ["</s>"]
    
print(sam_corpus)
    

['<s>', 'I', 'am', 'Sam', '.', '</s>', '<s>', 'Sam', 'I', 'am', '.', '</s>', '<s>', 'I', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '.', '</s>']


Now we make word pairs, and count. For this, we can use the function `nltk.bigrams()`:

In [3]:
sam_bigrams = list(nltk.bigrams(sam_corpus))

print(sam_bigrams)

[('<s>', 'I'), ('I', 'am'), ('am', 'Sam'), ('Sam', '.'), ('.', '</s>'), ('</s>', '<s>'), ('<s>', 'Sam'), ('Sam', 'I'), ('I', 'am'), ('am', '.'), ('.', '</s>'), ('</s>', '<s>'), ('<s>', 'I'), ('I', 'do'), ('do', 'not'), ('not', 'like'), ('like', 'green'), ('green', 'eggs'), ('eggs', 'and'), ('and', 'ham'), ('ham', '.'), ('.', '</s>')]


# Counting words, and computing relative frequencies

## Conditional Frequency Distribution objects in NLTK

The Natural Language Toolkit has data types and functions that make life easier for us when we want to count word n-grams and compute their probabilities.

A FreqDist object is a mapping from words (or word pairs) to counts. Here are the frequencies of all word pairs in the Sam corpus, sorted by frequency: 

In [4]:
fd = nltk.FreqDist(sam_bigrams)
for wordpair, count in fd.most_common(20):
    print(wordpair[0], wordpair[1], "\t", count)


. </s> 	 3
<s> I 	 2
I am 	 2
</s> <s> 	 2
am Sam 	 1
Sam . 	 1
<s> Sam 	 1
Sam I 	 1
am . 	 1
I do 	 1
do not 	 1
not like 	 1
like green 	 1
green eggs 	 1
eggs and 	 1
and ham 	 1
ham . 	 1


The ConditionalFreqDist object counts pairs of items, for example: Given that the first word in a pair is 'said', how often is the second word 'Alice'? How often is it 'the'?

In [5]:
# This makes the object
cfd = nltk.ConditionalFreqDist(sam_bigrams)
# This looks up the counts of bigrams starting in "I"
cfd["I"]

FreqDist({'am': 2, 'do': 1})

## Computing bigram probabilities 

As we have discussed in class, we can *estimate* the probability of, say, seeing the word "am" *given that* we have just seen "I", $P(am | I)$ using word counts: It is the number of times we have seen "I am", out of all the times we have seen "I" plus some following word. 

NLTK has a data structure that does this for us automatically: It converts a ConditionalFreqDist into probabilities. This data structure is, appropriately, called a ConditionalProbDist.

We tell it that we want "MLE" probabilities, which stands for "Maximum Likelihood Estimation". This is exactly what we have been discussing, estimating probabilities as relative frequencies.

In [6]:
# this makes the object
cp = nltk.ConditionalProbDist(cfd, nltk.MLEProbDist)
# this is p(am|I)
cp["I"].prob("am")

0.6666666666666666

In [7]:
# Here are all the words that could follow "I"
print("Words that can come after 'I':\n", list(cp["I"].samples()))
# For each word in samples(), we can get its probability
# using prob()
print("And here are their probabilities:")
for word in cp["I"].samples():
    print(word, cp["I"].prob(word))

Words that can come after 'I':
 ['am', 'do']
And here are their probabilities:
am 0.6666666666666666
do 0.3333333333333333


# Generating text: In each step, choose the next word proportional to its frequency 


We can now use the Sam corpus to generate text. This is just like predictive typing. In the simplest case, we can always select the most frequent following word. For "I", this is "am". (When there are several words with equal frequency as a next word, we'll just pick the first one for simplicity.) Here is what this does:

In [8]:
currentword = "<s>"
print(currentword, end = " ")

for i in range(30):
    currentword = cp[currentword].max()
    print(currentword, end = " ")

<s> I am Sam . </s> <s> I am Sam . </s> <s> I am Sam . </s> <s> I am Sam . </s> <s> I am Sam . </s> <s> 

This is a bit repetitive. We can generate more interesting text by not always picking the single most likely next word, but choosing among the possible next words by their frequency of following our current word -- or rather, choosing the next word by its conditionalm probability given the previous word. 

The Python *scipy* package has an object that describes a multinomial distribution, called ```stats.multinomial```. It has a function called ```rvs()``` that we can use for this purpose: It will give us a probabilistic answer, a different one each time we run it. 

In [9]:
from scipy import stats

num_words_to_generate = 1
# which words can follow "I"?
words_following_i = list(cp["I"].samples())
# and what are their probabilities? 
outcome_probs = [cp["I"].prob(word) for word in words_following_i]
print("the words are:", words_following_i)
print("with matching probabilities", outcome_probs)
print("Note that they sum up to one.")

# let's get one sample
outcome = stats.multinomial.rvs(num_words_to_generate, outcome_probs)
print("raw output of rvs:", outcome)

# not so very readable. 
# this gives us, for each word that can follow "I", 
# how often it was sampled. 
# here is how we can make it readable:
# at what position does this result have a "1"?
# that's the word we sampled
index_of_outcome = list(outcome).index(1)
# The new current word is the word at that index
sampled_word = words_following_i[ index_of_outcome]
print("we got this word:", sampled_word)
    

the words are: ['am', 'do']
with matching probabilities [0.6666666666666666, 0.3333333333333333]
Note that they sum up to one.
raw output of rvs: [1 0]
we got this word: am


In [10]:
# let's do this a few times, to see what we get
for i in range(20):
    outcome = stats.multinomial.rvs(num_words_to_generate, outcome_probs)
    index_of_outcome = list(outcome).index(1)
    # The new current word is the word at that index
    sampled_word = words_following_i[ index_of_outcome]
    print(sampled_word, end = " ")

am am am am am do do do do do am am am am am do do am am am 

Here is what `stats.multinomial` actually is:

Say you roll one die. If it is a fair die, rhen each side has a probability of 1/6 of coming up. We can describe this with a *categorical distribution*.

Now say you roll a loaded die. We can still describe this with a categorical distribution, the probabilities of all the sides are just not going to be the same.

Now say we roll 20 dice that are all fair (or unfair) in the same way. How often will we get a 1? This can be described with a *multinomial distribution*.

Here is a dice-rolling example. If you run this box several times, you should get a different result each time.

In [11]:

# we have 6 possible outcomes, each with a probability of 1/6
outcomes = [1,2,3,4,5,6]
outcome_probs = [1/6, 1/6, 1/6, 1/6, 1/6, 1/6]
# we only roll one die
num_dice = 1

result = stats.multinomial.rvs(num_dice, outcome_probs)
print("I just rolled one die. Here is what I got.")
# the result we get is a list of 0's and 1's. We get a 1 for
# the number that we rolled
print("Here is what the result looks like out of the box:", result)
# the index of the outcome we got is the 
# place in the result that has a 1
index_of_outcome = list(result).index(1)
print("And here is how we can interpret this:")
print("I rolled a", outcomes[index_of_outcome])


I just rolled one die. Here is what I got.
Here is what the result looks like out of the box: [0 0 1 0 0 0]
And here is how we can interpret this:
I rolled a 3


Now we are ready to generate a longer stretch of text. We start with the sentence start symbol.

In [12]:
currentword = "<s>"
print(currentword, end = " ")
# The language model knows, for each word w, 
# the conditional probability P(w|currentword).
# We use that to generate text:
# we start with the sentence-start symbol
for i in range(50):
    # get all the possible words that can follow the current word
    outcomes = list(cp[currentword].samples())
    # and get their probabilities
    outcome_probs = [cp[currentword].prob(word) for word in outcomes]
    # we only sample one next word
    num_dice = 1
    # sample the next word
    result = stats.multinomial.rvs(num_dice, outcome_probs)
    # which outcome index did we sample?
    index_of_outcome = list(result).index(1)
    # The new current word is the word at that index
    currentword = outcomes[ index_of_outcome]
    # print it, and repeat
    print(currentword,end = " ")

<s> I do not like green eggs and ham . </s> <s> I do not like green eggs and ham . </s> <s> I am Sam I do not like green eggs and ham . </s> <s> Sam . </s> <s> Sam . </s> <s> Sam I am . </s> <s> 

# Everything again, just much easier: NLTK's built-in language n-gram language model

NLTK also has a built-in language model with which we can do the same thing more easily:

In [13]:
# Let's re-preprocess the Sam corpus, 
# to remind ourselves how we did it.
# we import the natural language toolkit
import nltk


# Here is our corpus, first as plain text
text = "I am Sam. Sam I am. I do not like green eggs and ham."
# ... and here it is cut up into sentences
text_sents = nltk.sent_tokenize(text)
# ... and cut up into words
text_words = nltk.word_tokenize(text)
# ... and cut up into sentences, each of which
# is cut up into words
text_sent_words = [ nltk.word_tokenize(s) for s in text_sents]
    

Instead of only using bigrams to come up with word sequences, we can also add single-word (unigram) counts, so that we also sometimes generate word sequences that haven't been seen exactly like that in the text. NLTK has a fun function "everygrams" that computes n-grams up to a given length:

In [14]:
list(nltk.everygrams(text_sent_words[0], max_len  = 2))

[('I',), ('I', 'am'), ('am',), ('am', 'Sam'), ('Sam',), ('Sam', '.'), ('.',)]

In [15]:
# NLTK has a functionality for helping us with 
# the sentence tags
from nltk.lm.preprocessing import pad_both_ends
# the parameter n=2 tells the function
# that we're going to do bigrams eventually
list(pad_both_ends(text_sent_words[0], n=2))


['<s>', 'I', 'am', 'Sam', '.', '</s>']

In [16]:
# padding and making bigrams,
# then combining everything into a 
# list of words 
from nltk.lm.preprocessing import padded_everygram_pipeline
sam_corpus_thing, vocabulary_thing = padded_everygram_pipeline(2, text_sent_words)

# usually we don't turn this iterators
# into lists, but let's do it here
# to see what we've got
sam_corpus = [list(s) for s in sam_corpus_thing]
vocabulary = list(vocabulary_thing)

print(sam_corpus)
print(vocabulary)

[[('<s>',), ('<s>', 'I'), ('I',), ('I', 'am'), ('am',), ('am', 'Sam'), ('Sam',), ('Sam', '.'), ('.',), ('.', '</s>'), ('</s>',)], [('<s>',), ('<s>', 'Sam'), ('Sam',), ('Sam', 'I'), ('I',), ('I', 'am'), ('am',), ('am', '.'), ('.',), ('.', '</s>'), ('</s>',)], [('<s>',), ('<s>', 'I'), ('I',), ('I', 'do'), ('do',), ('do', 'not'), ('not',), ('not', 'like'), ('like',), ('like', 'green'), ('green',), ('green', 'eggs'), ('eggs',), ('eggs', 'and'), ('and',), ('and', 'ham'), ('ham',), ('ham', '.'), ('.',), ('.', '</s>'), ('</s>',)]]
['<s>', 'I', 'am', 'Sam', '.', '</s>', '<s>', 'Sam', 'I', 'am', '.', '</s>', '<s>', 'I', 'do', 'not', 'like', 'green', 'eggs', 'and', 'ham', '.', '</s>']


In [17]:
# preprocessing, and building a language model
sam_corpus, vocabulary = padded_everygram_pipeline(2, text_sent_words)

# and its maximum likelihood estimation (MLE) language model
from nltk.lm import MLE

# lm is now an object of type MLE, in particular
# a bigram language model, because we put the parameter 2
lm_sam = MLE(2)

lm_sam.fit(sam_corpus, vocabulary)

Now we can use this language model. Note that it also generates sentences starting with "ham" or "green", even though it has never seen that in the corpus. That is because of the mixed-in single-word counts.

In [18]:
for word in lm_sam.generate(50):
    print(word, end = " ")

</s> Sam . </s> . </s> . </s> <s> Sam . </s> am Sam I do not like green eggs and ham . </s> do not like green eggs and ham . </s> <s> Sam I do not like green eggs and ham . </s> green eggs and ham . 

## Getting more information out of the language model

Here is how to look up the conditional probability of, say "am" given "I": 

In [19]:
lm_sam.score("am", [ "I"])

0.6666666666666666

And here is the probability (non-conditional) for a word, here "am":

In [20]:
lm_sam.score("am")

0.08695652173913043

## Using a more interesting text

Let's finally move away from the tiny "Sam corpus". Let's use Lewis Carroll's "Alice in Wonderland", available on Project Gutenberg here: http://www.gutenberg.org/ebooks/11 Please get the plain text version, and store it in the same directory as this notebook as a file 11-0.txt

We read the data:

In [21]:
f = open("11-0.txt")
alice_text = f.read()
f.close()

import nltk

alice_sents = nltk.sent_tokenize(alice_text)
alice_sents_tokenized = [ nltk.word_tokenize(s) for s in alice_sents]


In [22]:
# preprocessing, which is now very simple,
# and building the language model
# let's do up to bigrams for now
ngrams = 2

alice_corpus, vocabulary = padded_everygram_pipeline(ngrams, alice_sents_tokenized)

lm_alice = MLE(ngrams)

lm_alice.fit(alice_corpus, vocabulary)

In [23]:
# and generating text
for word in lm_alice.generate(50):
    print(word, end = " ")

” said Alice could not uniform and feebly stretching out altogether , would be no use of sticks and washing ? ” said the Gryphon in it might not possibly make out in a little , it , as hard as she did not protected by wild beast , and 

In [24]:
# how are things different if we do trigrams?
ngrams = 3

alice_corpus, vocabulary = padded_everygram_pipeline(ngrams, alice_sents_tokenized)

lm_alice = MLE(ngrams)

lm_alice.fit(alice_corpus, vocabulary)

In [25]:
# and generating text
for word in lm_alice.generate(50):
    print(word, end = " ")

“ Are their heads . </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> 